<a href="https://colab.research.google.com/github/EddyGiusepe/Pandemia_COVID-19/blob/main/COVID_19_EddyGiusepe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <h2 align='center'>COVID-19</h2>


Cientista de Dados Jr: Eddy Giusepe Chirinos Isidro 


Neste Script reproduzimos os resultados do seguinte vídeo no [YouTube COVID-19](https://www.youtube.com/watch?v=Xk0zHZBa7LM).

## Importamos as nossas bibliotecas

In [1]:
import pandas as pd
import numpy as np 
import warnings
warnings.filterwarnings('ignore')



In [2]:
file = '/content/drive/MyDrive/5_Lira_minerando_etc/COVID-19_Eddy/covid_19_clean_complete.csv'
df = pd.read_csv(file, sep=',', parse_dates=['Date'])

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
df.head(10) 

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered
0,NaN,Afghanistan,33.0000,65.0000,2020-01-22,0,0,0
1,NaN,Albania,41.1533,20.1683,2020-01-22,0,0,0
2,NaN,Algeria,28.0339,1.6596,2020-01-22,0,0,0
3,NaN,Andorra,42.5063,1.5218,2020-01-22,0,0,0
4,NaN,Angola,-11.2027,17.8739,2020-01-22,0,0,0
5,NaN,Antigua and Barbuda,17.0608,-61.7964,2020-01-22,0,0,0
6,NaN,Argentina,-38.4161,-63.6167,2020-01-22,0,0,0
7,NaN,Armenia,40.0691,45.0382,2020-01-22,0,0,0
8,Australian Capital Territory,Australia,-35.4735,149.0124,2020-01-22,0,0,0
9,New South Wales,Australia,-33.8688,151.2093,2020-01-22,0,0,0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19836 entries, 0 to 19835
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Province/State  6080 non-null   object        
 1   Country/Region  19836 non-null  object        
 2   Lat             19836 non-null  float64       
 3   Long            19836 non-null  float64       
 4   Date            19836 non-null  datetime64[ns]
 5   Confirmed       19836 non-null  int64         
 6   Deaths          19836 non-null  int64         
 7   Recovered       19836 non-null  int64         
dtypes: datetime64[ns](1), float64(2), int64(3), object(2)
memory usage: 1.2+ MB


In [6]:
# Casos ativos = Casos Confirmados - Mortes - Casos Recoperados
df['Active'] = df['Confirmed'] - df['Deaths'] - df['Recovered']

In [7]:
# Subistituindo Mainland China por China
df['Country/Region'] = df['Country/Region'].replace('Mainland China', 'China')

In [8]:
# Preenchendo missing values 
df[['Province/State']] = df[['Province/State']].fillna('')
df[['Confirmed', 'Deaths', 'Recovered', 'Active']] = df[['Confirmed', 'Deaths', 'Recovered', 'Active']].fillna(0)

In [9]:
# Convertendo datatypes
df['Recovered'] = df['Recovered'].astype(int)

In [10]:
# Imprimindo 5 primeiras linhas, novamente.
df.head()

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered,Active
0,,Afghanistan,33.0000,65.0000,2020-01-22,0,0,0,0
1,,Albania,41.1533,20.1683,2020-01-22,0,0,0,0
2,,Algeria,28.0339,1.6596,2020-01-22,0,0,0,0
3,,Andorra,42.5063,1.5218,2020-01-22,0,0,0,0
4,,Angola,-11.2027,17.8739,2020-01-22,0,0,0,0


## Examinamos os Dados temporais

In [11]:
df.Date.describe()

count                   19836
unique                     76
top       2020-01-25 00:00:00
freq                      261
first     2020-01-22 00:00:00
last      2020-04-06 00:00:00
Name: Date, dtype: object

## Agrupando Dados



In [12]:
# Obtem o numero de casos confirmados, mortes, recuperados e ativos agrupado por data e por região.
df_agrupado = df.groupby(['Date', 'Country/Region'])['Confirmed', 'Deaths', 'Recovered', 'Active'].sum().reset_index()

In [13]:
# Ordena o dataframe por mais casos confirmados
df_agrupado.sort_values(by='Confirmed', ascending=False)

,Date,Country/Region,Confirmed,Deaths,Recovered,Active
13971,2020-04-06,US,366614,10783,19581,336250
13787,2020-04-05,US,337072,9619,17448,310005
13603,2020-04-04,US,308850,8407,14652,285791
13419,2020-04-03,US,275586,7087,9707,258792
13235,2020-04-02,US,243453,5926,9001,228526
...,...,...,...,...,...,...
8756,2020-03-09,Mauritania,0,0,0,0
4967,2020-02-17,Zimbabwe,0,0,0,0
8754,2020-03-09,Mali,0,0,0,0
4968,2020-02-18,Afghanistan,0,0,0,0


In [14]:
# Obtem o numero de casos confirmados, mortes, recuperados e ativos agrupando por região.
df_group_paises = df.groupby('Country/Region')['Confirmed', 'Deaths', 'Recovered', 'Active'].sum().reset_index()

In [15]:
# ordena por paises com mais casos confirmados
df_group_paises.sort_values(by='Confirmed', ascending=False)

,Country/Region,Confirmed,Deaths,Recovered,Active
36,China,4683417,165756,2847170,1670491
171,US,2831915,64777,98681,2668457
84,Italy,1942859,206052,266247,1470560
156,Spain,1472568,123441,295573,1053554
65,Germany,1142172,12016,217756,912400
...,...,...,...,...,...
103,Malawi,19,0,0,19
166,Timor-Leste,16,0,0,16
181,Western Sahara,8,0,0,8
143,Sao Tome and Principe,4,0,0,4


In [16]:
# Agrupa quantidade de casos recuperados, mortes e ativos por data
temp = df.groupby('Date')['Recovered', 'Deaths', 'Active'].sum().reset_index()

In [17]:
# Remodela o dataframe com variável e valor para ter quantidades de recuperados, mortos e ativos
temp = temp.melt(id_vars="Date", value_vars=['Recovered', 'Deaths', 'Active'],
                 var_name='Case', value_name='Count')

In [18]:
temp.head(20)

,Date,Case,Count
0,2020-01-22,Recovered,28
1,2020-01-23,Recovered,30
2,2020-01-24,Recovered,36
3,2020-01-25,Recovered,39
4,2020-01-26,Recovered,52
5,2020-01-27,Recovered,61
6,2020-01-28,Recovered,107
7,2020-01-29,Recovered,126
8,2020-01-30,Recovered,143
9,2020-01-31,Recovered,222


In [19]:
# habilita modo offline
from plotly.offline import plot, iplot, init_notebook_mode
init_notebook_mode(connected=True)

In [20]:
# Definindo o renderizador:
import plotly.io as pio
pio.renderers
pio.renderers.default = "colab"

In [21]:
# Cores
recuperados = '#21bf73'
mortes = '#ff2e63'
ativos = '#fe9801'

In [22]:
import plotly.express as px
fig = px.area(temp, 
              x="Date", 
              y="Count", 
              color='Case', 
              height=600,
              title='Casos ao longo do tempo',
              color_discrete_sequence = [recuperados, mortes, ativos])
fig.update_layout(xaxis_rangeslider_visible=True)
fig.show()

## Casos ao longo do tempo

In [23]:
# Mapa de Choropleth é um mapa composto por polígonos coloridos. 
# É usado para representar variações espaciais de uma quantidade
fig = px.choropleth(df_agrupado,                                                   # casos agrupados por país
                    locations="Country/Region",                                    # definindo as regiões no mapa
                    locationmode='country names',                                  # define o modo de localização para todas regiões
                    color=np.log(df_agrupado["Confirmed"]),                        # define a cor pelo o valor de casos confirmados (aplica o log)
                    hover_name='Country/Region',                                   # define o texto interativo com o nome da região
                    hover_data=["Confirmed", "Deaths"],                            # define o texto interativo com o numero de casos confirmasos e mortes
                    animation_frame=df_agrupado["Date"].dt.strftime('%d-%m-%Y'),   # define o animate_frame com as datas
                    title='Casos ao longo do tempo',                               # define título
                    color_continuous_scale=px.colors.sequential.Magenta)           # define a paleta de cores
fig.update_layout(autosize=False, width=1200, height=800)                          # define tamanho da figura
fig.show()

## Mortes ao longo do tempo

In [24]:
# Mapa de Choropleth é um mapa composto por polígonos coloridos. 
# É usado para representar variações espaciais de uma quantidade
fig = px.choropleth(df_agrupado,                                                   # casos agrupados por país
                    locations="Country/Region",                                    # definindo as regiões no mapa
                    locationmode='country names',                                  # define o modo de localização para todas regiões
                    color=np.log(df_agrupado["Deaths"]),                        # define a cor pelo o valor de casos confirmados (aplica o log)
                    hover_name='Country/Region',                                   # define o texto interativo com o nome da região
                    hover_data=["Confirmed", "Deaths"],                            # define o texto interativo com o numero de casos confirmasos e mortes
                    animation_frame=df_agrupado["Date"].dt.strftime('%d-%m-%Y'),   # define o animate_frame com as datas
                    title='Mortes ao longo do tempo',                               # define título
                    color_continuous_scale=px.colors.sequential.Magenta)           # define a paleta de cores
fig.update_layout(autosize=False, width=1200, height=800)
fig.show()

## Painel

In [25]:
!pip install plotly==4.5.2

In [26]:
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
import plotly.express as px

In [27]:
# filtra os dados considerando o último dia da base de dados
completo = df[df['Date'] == max(df['Date'])]

In [28]:
# imprime as 5 primeiras linhas
completo.head()

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered,Active
19575,,Afghanistan,33.0000,65.0000,2020-04-06,367,11,18,338
19576,,Albania,41.1533,20.1683,2020-04-06,377,21,116,240
19577,,Algeria,28.0339,1.6596,2020-04-06,1423,173,90,1160
19578,,Andorra,42.5063,1.5218,2020-04-06,525,21,31,473
19579,,Angola,-11.2027,17.8739,2020-04-06,16,2,2,12


In [29]:
# Plota painel                        
fig = px.treemap(completo.sort_values(by='Confirmed', ascending=False).reset_index(drop=True), 
                 path=["Country/Region", "Province/State"],
                 values="Confirmed",
                 height=600,
                 title='Número de Casos Confirmados',
                 color_discrete_sequence = px.colors.qualitative.Dark2)
fig.data[0].textinfo = 'label+text+value'
fig.show()
# Plota Painel
fig = px.treemap(completo.sort_values(by='Deaths', ascending=False).reset_index(drop=True), 
                 path=["Country/Region", "Province/State"],
                 values="Deaths",
                 height=600,
                 title='Número de Mortes Confirmadas',
                 color_discrete_sequence = px.colors.qualitative.Dark2)
fig.data[0].textinfo = 'label+text+value'
fig.show()

## Picos de casos confirmados e mortes

In [30]:
fig = px.line(df_agrupado,
              x="Date",
              y="Confirmed",
              color='Country/Region',
              height=600,
              title='Casos Confirmados',
              color_discrete_sequence = px.colors.qualitative.Dark2 )
fig.show()

fig = px.line(df_agrupado,
              x="Date",
              y="Deaths",
              color='Country/Region',
              height=600,
              title='Mortes Confirmadas',
              color_discrete_sequence = px.colors.qualitative.Dark2)
fig.show()

Gráfico com Folium

In [31]:
# Obtem os dados do último dia da base de dados
temp = df[df['Date'] == max(df['Date'])]

In [32]:
import folium

In [33]:
m = folium.Map(location=[0, 0], tiles='cartodbpositron',
               min_zoom=1, max_zoom=4, zoom_start=1)

for i in range(0, len(temp)):
    folium.Circle(
        location=[temp.iloc[i]['Lat'], temp.iloc[i]['Long']],
        color='crimson', fill='crimson',
        tooltip =   '<li><bold>Country : '+str(temp.iloc[i]['Country/Region'])+
                    '<li><bold>Province : '+str(temp.iloc[i]['Province/State'])+
                    '<li><bold>Confirmed : '+str(temp.iloc[i]['Confirmed'])+
                    '<li><bold>Deaths : '+str(temp.iloc[i]['Deaths']),
        radius=int(temp.iloc[i]['Confirmed'])**1.1).add_to(m)
m